In [1]:
%cd /content/
!git clone https://github.com/kikirizki/PocketNet
!gdown 1sDnU-DuotD0nkJ_1AitlqZGu1Torwifu

/content
Cloning into 'PocketNet'...
remote: Enumerating objects: 622, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 622 (delta 32), reused 30 (delta 27), pack-reused 576
Receiving objects: 100% (622/622), 73.18 MiB | 36.09 MiB/s, done.
Resolving deltas: 100% (346/346), done.
Downloading...
From: https://drive.google.com/uc?id=1sDnU-DuotD0nkJ_1AitlqZGu1Torwifu
To: /content/295672backbone.pth
100% 4.24M/4.24M [00:00<00:00, 212MB/s]


In [2]:
%cd /content/PocketNet
import logging
import os
import sys
import torch
from backbones.augment_cnn import AugmentCNN 
import backbones.genotypes as gt

from configs.config_PocketNetS128 import config as cfg



genotype = gt.from_str(cfg.genotypes["softmax_casia"])
backbone=AugmentCNN(C=cfg.channel, n_layers=cfg.n_layers, genotype=genotype, stem_multiplier=4,
                       emb=cfg.embedding_size)
backbone.load_state_dict(torch.load("/content/295672backbone.pth"))
backbone.eval()
x = torch.randn([1,3,112,112])
torch.onnx.export(backbone,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "pocketnet.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                 )

/content/PocketNet
call Factorized
call Factorized


In [3]:
%cd /content/
!wget -q https://github.com/Tencent/ncnn/releases/download/20220216/ncnn-20220216-ubuntu-1804.zip
!unzip -q ncnn-20220216-ubuntu-1804.zip

import os
os.environ['PATH'] += ":/content/ncnn-20220216-ubuntu-1804/bin"

/content


In [4]:
!onnx2ncnn \
  PocketNet/pocketnet.onnx \
  pocketnet.param \
  pocketnet.bin

In [5]:
!apt install vulkan-utils
# https://github.com/Tencent/ncnn/wiki/use-ncnnoptimize-to-optimize-model
!ncnnoptimize \
  pocketnet.param \
  pocketnet.bin \
  pocketnet-opt.param \
  pocketnet-opt.bin \
  65536

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers vulkan-tools
The following NEW packages will be installed:
  libvulkan1 mesa-vulkan-drivers vulkan-tools vulkan-utils
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 3,315 kB of archives.
After this operation, 15.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 mesa-vulkan-drivers amd64 20.0.8-0ubuntu1~18.04.1 [3,059 kB]
Get:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 libvulkan1 amd64 1.1.126.0-2~gpu18.04.1 [101 kB]
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 vulkan-tools amd64 1.1.126.0+dfsg1-1~gpu18.04.1 [152 kB]
Get:4 ht

In [6]:
!mkdir pocketnet
!cp pocketnet-opt.param pocketnet/pocketnet.param
!cp pocketnet-opt.bin pocketnet/pocketnet.bin

!zip -r pocketnet.zip pocketnet

from google.colab import files
files.download("pocketnet.zip")

  adding: pocketnet/ (stored 0%)
  adding: pocketnet/pocketnet.param (deflated 88%)
  adding: pocketnet/pocketnet.bin (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>